In [1]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from lightgbm import LGBMClassifier

In [2]:
train = pd.read_csv('/kaggle/input/jigsaw-agile-community-rules/train.csv')

In [3]:
test = pd.read_csv('/kaggle/input/jigsaw-agile-community-rules/test.csv')

In [4]:
test

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
0,2029,NEW RAP GROUP 17. CHECK US OUT https://soundcl...,"No Advertising: Spam, referral links, unsolici...",hiphopheads,"Hey, guys, just wanted to drop in and invite y...",Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng - [Chelsea TV USA](http://soccer...,HD Streams: |[ENG HD Stoke vs Manchester Unite...
1,2030,Make your life comfortable. Get up to 15% Disc...,No legal advice: Do not offer or request legal...,AskReddit,Get a lawyer and get the security camera foota...,That isn't drastic. You tried reaching out to ...,So what are you going to do with the insurance...,It's just for Austria & Germany. If you still ...
2,2031,Kickin' ass and selling underwear!\nJust made ...,"No Advertising: Spam, referral links, unsolici...",gonewild,Good story my friend. Check out my blog at ht...,If you know what exactly you need then you don...,CENTIPEDES\n\nSOME BASED PATRIOTS HAVE CREATED...,[So great! Thanks for sharing.](http://www.che...
3,2032,watch hooters best therein http://clickan...,"No Advertising: Spam, referral links, unsolici...",personalfinance,"Earn 50,000 bonus points with Chase Sapphire P...","Cool, front page! I made this print along with...",[Full HD Movie Online Free](http://www.flickma...,* Karambit Black Pearl\n* 0.02137822 Float (un...
4,2033,bitches for free at this point show all h...,"No Advertising: Spam, referral links, unsolici...",Showerthoughts,code free tyrande --->>> [Imgur](http://i.imgu...,My trade link\nhttps://steamcommunity.com/trad...,**HD** [ mio Stadium 102 HD](http://www.genti....,Infographics is an incredible method for showi...
5,2034,Top 10 Pentakills in August https://youtu.be/Z...,"No Advertising: Spam, referral links, unsolici...",leagueoflegends,code free tyrande --->>> [Imgur](http://i.imgu...,see tits sweet at this point http://sh.or...,its my asreddit https://www.youtube.com/watch?...,click here for more videos[Russian mafia messe...
6,2035,WIN A SAMSUNG SMARTWATCH!\n\nhttp://www.innomd...,"No Advertising: Spam, referral links, unsolici...",AskReddit,"[ENJOY 50,000 BONUS POINTS after $4,000 spent ...",We have high quality Canadian Polymer banknote...,i was on the same ios and try this jailbreak m...,We're streaming Pokemon Veitnamese Crystal RIG...
7,2036,[This mixtape is lit FAM!](https://beholdthear...,"No Advertising: Spam, referral links, unsolici...",BlackPeopleTwitter,watch good one hooters there http://https://x5...,bad pussies waiting for you herein http://gour...,[Full HD Movie Online Free](http://www.flickma...,* Computers :http://livematchstreamz.blogspot....
8,2037,must be watch movie https://sites.google.com/s...,"No Advertising: Spam, referral links, unsolici...",movies,see there hooters sweet http://url.t.aylo...,"Sign up with my link and get $1,000 of free pr...",The fact that there are pro and cons to everyt...,Oh that's interesting. You should check out my...
9,2038,Free paypal cards here!! https://www.pointspri...,"No Advertising: Spam, referral links, unsolici...",pics,she will come your home open her legs with an...,Old and young your girl-next-door make sex wi...,"i think the conjuring 2 is not bad movie ,The...","wow, she's so beautyful\n>[Sakura Vietnam](htt..."


In [5]:
train.head()

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1


## 데이터 나누기

In [6]:
def df_pos_split(df):
    positive_df1 = df['positive_example_1']
    positive_df2 = df['positive_example_2']
    
    positive_df = pd.concat([positive_df1, positive_df2], ignore_index=True).to_frame(name="body")
    positive_df['target'] = 1

    return positive_df

In [7]:
def df_neg_split(df):
    negative_df1 = df['negative_example_1']
    negative_df2 = df['negative_example_2']
    
    negative_df = pd.concat([negative_df1, negative_df2], ignore_index=True).to_frame(name="body")
    negative_df['target'] = 0
    
    return negative_df

In [8]:
tr_df_pos = df_pos_split(train)
tr_df_neg = df_neg_split(train)

ts_df_pos = df_pos_split(test)
ts_df_neg = df_neg_split(test)

In [9]:
tr_df = pd.concat([tr_df_pos, tr_df_neg], ignore_index=True)
ts_df = pd.concat([ts_df_pos, ts_df_neg], ignore_index=True)
train_df = pd.concat([tr_df, ts_df], ignore_index=True)

train_df.head()

,body,target
0,If you could tell your younger self something ...,1
1,[I wanna kiss you all over! Stunning!](http://...,1
2,Don't break up with him or call the cops. If ...,1
3,Selling Tyrande codes for 3€ to paypal. PM. \n...,1
4,wow!! amazing reminds me of the old days.Well...,1


In [10]:
train_df

,body,target
0,If you could tell your younger self something ...,1
1,[I wanna kiss you all over! Stunning!](http://...,1
2,Don't break up with him or call the cops. If ...,1
3,Selling Tyrande codes for 3€ to paypal. PM. \n...,1
4,wow!! amazing reminds me of the old days.Well...,1
...,...,...
8151,click here for more videos[Russian mafia messe...,0
8152,We're streaming Pokemon Veitnamese Crystal RIG...,0
8153,* Computers :http://livematchstreamz.blogspot....,0
8154,Oh that's interesting. You should check out my...,0


In [11]:
t_df = train[['body', 'rule_violation']]
t_df = t_df.rename(columns={
    'rule_violation': 'target'
})
t_df

,body,target
0,Banks don't want you to know this! Click here ...,0
1,SD Stream [ ENG Link 1] (http://www.sportsstre...,0
2,Lol. Try appealing the ban and say you won't d...,1
3,she will come your home open her legs with an...,1
4,code free tyrande --->>> [Imgur](http://i.imgu...,1
...,...,...
2024,Please edit your post so it is readable. These...,1
2025,"Yes, and in a right to work state they can eve...",0
2026,**HD** Streams: |ENG **HD**[ Watch here..PC & ...,1
2027,No. Not when doing so obviously presents a saf...,1


In [12]:
train_df = pd.concat([train_df, t_df], ignore_index=True)
train_df

,body,target
0,If you could tell your younger self something ...,1
1,[I wanna kiss you all over! Stunning!](http://...,1
2,Don't break up with him or call the cops. If ...,1
3,Selling Tyrande codes for 3€ to paypal. PM. \n...,1
4,wow!! amazing reminds me of the old days.Well...,1
...,...,...
10180,Please edit your post so it is readable. These...,1
10181,"Yes, and in a right to work state they can eve...",0
10182,**HD** Streams: |ENG **HD**[ Watch here..PC & ...,1
10183,No. Not when doing so obviously presents a saf...,1


In [13]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", " ", text)       # URL 제거
    text = re.sub(r"[^a-zA-Z\s]", " ", text)   # 알파벳 외 제거 (한국어면 따로 tokenizer 필요)
    text = re.sub(r"\s+", " ", text).strip()   # 공백 정리
    return text

train_df["body_clean"] = train_df["body"].apply(clean_text)

In [14]:
vectorizer = TfidfVectorizer(
    max_features=5000,      # 상위 5000 단어
    ngram_range=(1,2),      # unigram + bigram
    stop_words="english"    # 불용어 제거 (한국어면 따로 stopwords 설정)
)
X = vectorizer.fit_transform(train_df["body_clean"])

In [15]:
y = train_df["target"]

In [16]:
lgbm = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    random_state=42,
    class_weight="balanced"
)

lgbm.fit(X, y)

[LightGBM] [Info] Number of positive: 5109, number of negative: 5076
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.077980 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11894
[LightGBM] [Info] Number of data points in the train set: 10185, number of used features: 1787
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


LGBMClassifier(class_weight='balanced', learning_rate=0.05, n_estimators=500,
               random_state=42)

In [17]:
test["body_clean"] = test["body"].apply(clean_text)

In [18]:
X_test = vectorizer.transform(test["body_clean"])

In [19]:
test_pred = lgbm.predict_proba(X_test)[:, 1]

In [20]:
test_pred

array([0.07646699, 0.50352367, 0.9953719 , 0.99787073, 0.9834361 ,
       0.0264477 , 0.95496491, 0.16329004, 0.02050146, 0.96546213])

In [21]:
submission_df = pd.DataFrame({'row_id': test['row_id'], 'rule_violation': test_pred})
submission_df.to_csv('submission.csv', index=False)